In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
directory = '/content/drive/My Drive/Colab Notebooks/raw_data'
training_directory = '/content/drive/My Drive/Colab Notebooks/raw_data/Training'
testing_directory = '/content/drive/My Drive/Colab Notebooks/raw_data/Testing'

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

train_ds, val_ds = image_dataset_from_directory(
  training_directory,
  labels = "inferred",
  label_mode = "categorical",
  seed=123,
  image_size=(255, 255),
  validation_split = 0.2,
  subset="both",
  color_mode = 'rgb',
  batch_size= 64)

test_ds = image_dataset_from_directory(
  testing_directory,
  labels = "inferred",
  label_mode = "categorical",
  seed=123,
  image_size=(255, 255),
  color_mode = 'rgb',
  batch_size= 64)

Found 5712 files belonging to 4 classes.
Using 4570 files for training.
Using 1142 files for validation.
Found 1311 files belonging to 4 classes.


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50
from tensorflow.keras.models import Model


def load_model():
    
    model = ResNet50(weights='imagenet', input_shape = (255, 255, 3), include_top = False)

    inputs = layers.Input(shape = (255, 255, 3))

    x = preprocess_input(inputs)

    x = model(x)

    model = Model(inputs = inputs , outputs = x)
    
    return model

In [ ]:
model = load_model()

In [ ]:
model.summary()

Model: "model_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_50 (InputLayer)       [(None, 255, 255, 3)]     0         
                                                                 
 tf.__operators__.getitem_24  (None, 255, 255, 3)      0         
  (SlicingOpLambda)                                              
                                                                 
 tf.nn.bias_add_24 (TFOpLamb  (None, 255, 255, 3)      0         
 da)                                                             
                                                                 
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
def set_nontrainable_layers(model):
    
    model.trainable = False
    
    return model

In [ ]:
from tensorflow.keras import layers, models

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    
    base_model = load_model()
    base_model = set_nontrainable_layers(base_model)
    
    flattening_layer = layers.Flatten()
    
    dense_layer_1 = layers.Dense(100, activation='relu')
    dropout_layer_1 = layers.Dropout(rate = 0.5)

    prediction_layer = layers.Dense(4, activation='softmax')

    model = models.Sequential([
      base_model,
      flattening_layer,
      dense_layer_1,
      dropout_layer_1,
      prediction_layer
    ])
    
    return model

In [ ]:
from tensorflow.keras import optimizers

def build_model():
    
    model = load_model()

    model = add_last_layers(model)

    opt = optimizers.Adam(learning_rate=1e-4)

    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model = build_model()

es = EarlyStopping(patience = 10,
                   monitor ='val_accuracy',
                   restore_best_weights =True
                   )

history = model.fit(train_ds,
          batch_size=64,
          epochs=100,
          callbacks=es,
          validation_data=val_ds)

Epoch 1/100
38/72 [==============>...............] - ETA: 3:05 - loss: 0.9206 - accuracy: 0.7459

In [ ]:
import matplotlib.pyplot as plt

def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label='train' + exp_name)
    ax1.plot(history.history['val_loss'], label='val' + exp_name)
    #ax1.set_ylim(0., 2.2)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
    #ax2.set_ylim(0.25, 1.)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

In [ ]:
plot_history(history)

In [ ]:
res = model.evaluate(test_ds)
test_accuracy = res[-1]
print(f"test_accuracy = {round(test_accuracy,2)*100} %")

In [ ]:
results_1 : (91, 71), (91, 75), (91, 72)